In [7]:
import numpy as np
import pandas as pd
import pyarrow as pa
import os
import swat

In [8]:
os.environ['CAS_CLIENT_SSL_CA_LIST'] = '/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem'
#os.environ['CAS_CLIENT_SSL_CA_LIST'] = '/mnt/c/Users/frasep/OneDrive - SAS/Mes Documents/Dossiers/Demoing/My_Azure_Environments/terraform_monoserver/cacerts/'

cashost='frasepviya35smp.cloud.com'
casport=5570
s = swat.CAS(cashost, casport)
s.sessionProp.setSessOpt(timeout=1800, caslib='casuser', messageLevel="ALL", metrics='true')

NOTE: 'CASUSER(viyademo01)' is now the active caslib.
NOTE: Action 'sessionProp.setSessOpt' used (Total process time):
NOTE:       real time               0.000306 seconds
NOTE:       cpu time                0.000290 seconds (94.77%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  262.06K (0.00%)


+ Elapsed: 0.000306s, user: 0.000182s, sys: 0.000108s, mem: 0.256mb

In [4]:
df = pd.DataFrame({'one': [-1, np.nan, 2.5],
                   'two': ['foo', 'bar', 'baz'],
                   'three': [True, False, True]},
                    index=list('abc'))


table = pa.Table.from_pandas(df)

In [2]:
import pyarrow.parquet as pq

In [6]:
pq.write_table(table, 'c://my_local_data/example.parquet')

In [9]:
s.table.addcaslib(name="AzureDL", dataSource={"srctype":"adls", "accountname":'frasepstorage',"filesystem":"datalake", "tenantid":"b1c14d5c-3625-45b3-a430-9552373a0c2f","applicationId":"e7af42e8-3ca8-47bb-97ce-ac764019be3a","timeout":"50000"}, path="/", subDirectories="True", session="True")

NOTE: Executing action 'table.addCaslib'.
NOTE: 'AzureDL' is now the active caslib.
NOTE: Cloud Analytic Services added the caslib 'AzureDL'.
NOTE: Action 'table.addCaslib' used (Total process time):
NOTE:       real time               0.001008 seconds
NOTE:       cpu time                0.000972 seconds (96.43%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  765.13K (0.00%)


,Name,Type,Description,Path,Definition,Subdirs,Local,Active,Personal,Hidden,Transient
0,AzureDL,adls,,/,timeout = 50000,1.0,1.0,1.0,0.0,0.0,0.0
1,AzureDL,,,,accountName = 'frasepstorage',NaN,NaN,NaN,NaN,NaN,NaN
2,AzureDL,,,,fileSystem = 'datalake',NaN,NaN,NaN,NaN,NaN,NaN
3,AzureDL,,,,tenantId = 'b1c14d5c-3625-45b3-a430-9552373a0c2f',NaN,NaN,NaN,NaN,NaN,NaN
4,AzureDL,,,,applicationId = 'e7af42e8-3ca8-47bb-97ce-ac764...,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
s.table.loadtable(caslib="AzureDL", path="megacorp5_4m.orc", casout="megacorp5_4m");

NOTE: Executing action 'table.loadTable'.
NOTE: Cloud Analytic Services made the external data from megacorp5_4m.orc available as table MEGACORP5_4M in caslib AzureDL.
NOTE: Action 'table.loadTable' used (Total process time):
NOTE:       real time               20.469345 seconds
NOTE:       cpu time                17.041895 seconds (83.26%)
NOTE:       total nodes             1 (16 cores)
NOTE:       total memory            125.75G
NOTE:       memory                  31.23M (0.02%)


In [11]:
# data file
indata = 'megacorp5_4m'

# CASTable view
castbl = s.CASTable(indata,caslib='Azuredl')

In [14]:
my_tab = castbl.to_frame(table=dict(name=indata, caslib='Azuredl'))

In [18]:
pq.write_table(my_tab, '/mnt/demo/dnfs/megacorp5_4m.parquet',use_dictionary='false')

In [21]:
stream = os.popen("du -sh /mnt/demo/dnfs/megacorp5_4m.parquet")
output = stream.read()
print(output)

113M	/mnt/demo/dnfs/megacorp5_4m.parquet



In [25]:
my_tab.to_parquet('/mnt/demo/dnfs/megacorp5_4m.parquet')
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_parquet.html

In [22]:
my_tab.columns

Index(['Date', 'DateByYear', 'DateByMonth', 'DayOfWeek', 'FacilityID',
       'Facility', 'FacilityType', 'FacilityAge', 'EmployeesUsed',
       'FacilityRegion', 'FacilityState', 'FacilityCity', 'UnitID', 'Unit',
       'UnitDowntime', 'UnitAge', 'UnitLifespan', 'UnitLifespanLimit',
       'UnitReliability', 'UnitStatus', 'UnitCapacity', 'UnitYieldTarget',
       'UnitYieldActual', 'UnitYieldRate', 'ProductBrand', 'ProductLine',
       'ProductID', 'Product', 'ProductDescription', 'ProductPriceTarget',
       'ProductPriceActual', 'ProductMaterialCost', 'ProductQuality', 'Profit',
       'Revenue', 'Expenses', 'ExpensesCapital', 'ExpensesOperational',
       'ExpensesMaterial', 'ExpensesStaffing', 'RegionLongitude',
       'RegionLatitude', 'StateLongitude', 'StateLatitude', 'CityLongitude',
       'CityLatitude'],
      dtype='object')

In [23]:
type(my_tab)

swat.dataframe.SASDataFrame

In [26]:
s.close()